# Day19_3: OpenAI 파인튜닝 - LLM을 내 목적에 맞게 맞춤 제작하기

## 학습 목표

**Part 1: 기초**
1. 파인튜닝(Fine-Tuning)의 개념을 이해하고, 프롬프트 엔지니어링, RAG와의 차이점을 설명할 수 있습니다.
2. 파인튜닝이 적합한 시나리오와 부적합한 시나리오를 구별할 수 있습니다.
3. OpenAI 파인튜닝을 위한 JSONL 데이터 형식을 이해하고 올바르게 작성할 수 있습니다.
4. 고품질 학습 데이터의 특성을 이해하고, 데이터 준비 과정을 수행할 수 있습니다.
5. OpenAI Files API를 사용하여 학습 데이터를 업로드할 수 있습니다.

**Part 2: 심화**
1. OpenAI Fine-tuning API를 사용하여 파인튜닝 작업을 생성하고 모니터링할 수 있습니다.
2. 파인튜닝된 모델을 호출하고, 기본 모델과의 성능 차이를 비교 분석할 수 있습니다.
3. 하이퍼파라미터(epochs, learning_rate_multiplier)를 조정하여 학습을 최적화할 수 있습니다.
4. 실무에서 파인튜닝을 활용하는 전략과 비용 최적화 방법을 이해합니다.

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 파인튜닝 개념 | 도메인 특화 AI 구축 | 법률 문서 초안 작성, 브랜드 톤앤매너 유지 |
| Prompting vs RAG vs Fine-tuning | 최적 기술 선택 | 비용-성능 트레이드오프 분석 |
| JSONL 데이터 형식 | 학습 데이터 준비 | 대화 데이터셋 구축 |
| 파인튜닝 API | 맞춤형 모델 제작 | 고객 응대 챗봇, 코드 리뷰 봇 |

**분석가 관점**: 프롬프트 엔지니어링이나 RAG로 해결하기 어려운 일관된 스타일, 특정 출력 형식, 도메인 언어 습득이 필요한 경우 파인튜닝이 강력한 솔루션입니다. 한 번 학습시키면 짧은 프롬프트로도 높은 품질의 결과를 얻을 수 있어 API 비용을 절감할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 파인튜닝(Fine-Tuning)이란?

### LLM의 행동 자체를 변화시키는 기술

**파인튜닝(Fine-Tuning)** 또는 **미세조정**은 기존의 사전학습된 LLM을 우리만의 데이터로 추가 학습시켜, 모델의 **내부 가중치(weights)**를 업데이트하는 과정입니다.

| 특성 | 설명 |
|------|------|
| **핵심 아이디어** | 모델의 '뇌'를 직접 재훈련 |
| **변화 대상** | 모델 자체의 행동 방식, 스타일, 출력 패턴 |
| **필요 데이터** | 고품질 예시 데이터 (입력-출력 쌍) |
| **비유** | 천재 배우에게 특정 캐릭터 연기 훈련시키기 |

### 파인튜닝의 효과

```
기본 모델 (GPT-4o):
  - 범용적, 다재다능
  - 어떤 질문에도 답변 가능
  - 하지만 '우리 회사 스타일'로 답하지는 않음

파인튜닝된 모델:
  - 특정 도메인/스타일에 최적화
  - 짧은 프롬프트로도 원하는 출력
  - 일관된 품질 유지
```

### 파인튜닝이 적합한 상황

| 적합한 경우 | 부적합한 경우 |
|------------|-------------|
| 일관된 **스타일/톤앤매너** 필요 | 단순히 새로운 **지식** 추가 필요 |
| 특정 **출력 형식** 고정 (JSON, 특정 포맷) | 최신 정보가 계속 변경됨 |
| **도메인 언어/전문 용어** 학습 | 고품질 학습 데이터가 부족함 |
| 복잡한 지시를 **짧은 프롬프트**로 처리 | 빠른 프로토타이핑이 필요함 |
| **API 비용 절감** (긴 프롬프트 대체) | 비용 제약이 매우 큼 |

> **핵심 원칙**:  
> 새로운 '**지식**'을 가르쳐야 한다면 **RAG**를,  
> 모델의 '**행동 방식**'을 바꾸고 싶다면 **파인튜닝**을 선택하세요.

---

## 1.2 Prompting vs RAG vs Fine-tuning 비교

### 세 가지 LLM 최적화 기법

LLM을 특정 작업에 맞게 조정하는 대표적인 세 가지 방법을 비교해봅시다.

| 구분 | **프롬프트 엔지니어링** | **RAG (검색 증강 생성)** | **파인튜닝 (Fine-Tuning)** |
| :--- | :--- | :--- | :--- |
| **핵심 아이디어** | '지시사항(프롬프트)'을 정교하게 작성 | 외부 지식을 '참고 자료'로 제공 | 모델의 '내부 가중치'를 업데이트 |
| **모델 변화** | 없음 | 없음 | **있음** (재학습) |
| **적합한 상황** | 간단한 작업, 빠른 프로토타입 | 최신 정보, 사실 기반 답변 | 스타일/톤/형식 일관성 |
| **장점** | 빠르고 저렴, 즉시 적용 | 지식 업데이트 용이, 환각 감소 | 짧은 프롬프트로 높은 성능 |
| **단점** | 긴 프롬프트 필요, 일관성 부족 | 검색 시스템 구축 필요 | 고품질 데이터 필요, 비용 발생 |

In [ ]:
# 세 가지 접근법 비교 시각화
comparison = {
    "기법": ["프롬프트 엔지니어링", "RAG", "파인튜닝"],
    "구현 난이도": ["낮음", "중간", "높음"],
    "초기 비용": ["없음", "중간 (벡터DB)", "높음 (학습)"],
    "운영 비용": ["높음 (긴 프롬프트)", "중간 (검색+생성)", "낮음 (짧은 프롬프트)"],
    "지식 업데이트": ["즉시", "쉬움", "재학습 필요"],
    "일관성": ["낮음", "중간", "높음"]
}

print("=" * 70)
print("LLM 최적화 기법 비교")
print("=" * 70)
for key, values in comparison.items():
    print(f"\n{key}:")
    for i, val in enumerate(values):
        print(f"  - {comparison['기법'][i]}: {val}")

### 실무에서의 선택 가이드

```
질문 1: "최신 정보나 회사 내부 데이터가 필요한가?"
  -> YES: RAG 우선 고려
  -> NO: 다음 질문으로

질문 2: "일관된 스타일/형식/어조가 필요한가?"
  -> YES: 파인튜닝 고려
  -> NO: 프롬프트 엔지니어링으로 충분

질문 3: "고품질 예시 데이터가 충분한가? (최소 50개 이상)"
  -> YES: 파인튜닝 진행
  -> NO: Few-shot 프롬프팅으로 시작
```

> **실무 팁**: 세 가지 기법은 상호 보완적입니다. 파인튜닝된 모델 + RAG 조합도 가능합니다!

---

## 1.3 JSONL 데이터 형식 이해하기

### JSONL (JSON Lines) 형식

OpenAI 파인튜닝은 **JSONL** 형식의 데이터를 요구합니다. JSONL은 파일의 각 줄(line)이 하나의 독립된 JSON 객체인 형태입니다.

```json
{"messages": [{"role": "system", "content": "시스템 메시지"}, {"role": "user", "content": "질문"}, {"role": "assistant", "content": "이상적인 답변"}]}
{"messages": [{"role": "system", "content": "시스템 메시지"}, {"role": "user", "content": "두 번째 질문"}, {"role": "assistant", "content": "두 번째 답변"}]}
```

### 메시지 역할 구조

| 역할 | 필수 여부 | 설명 |
|------|----------|------|
| `system` | 선택 | 모델의 행동 방식, 역할 정의 |
| `user` | **필수** | 사용자의 입력/질문 |
| `assistant` | **필수** | 모델이 학습할 이상적인 답변 |

In [ ]:
# JSONL 형식 예시
import json

# 단일 학습 예시
training_example = {
    "messages": [
        {"role": "system", "content": "당신은 IT 전문 기자입니다. 뉴스 기사를 '핵심 요약'과 '주요 배경'으로 구조화하여 요약합니다."},
        {"role": "user", "content": "다음 뉴스 기사를 요약해 주세요:\n\n제목: AI 신약 개발 속도 10배 앞당긴다\n내용: 인공지능이 신약 개발 분야에서 혁신을 주도하고 있다..."},
        {"role": "assistant", "content": "- 핵심 요약: AI가 신약 개발 기간을 10년에서 1-2년으로 단축시키고 있습니다.\n- 주요 배경: AI 시뮬레이션과 데이터 분석을 통해 유효 물질을 빠르게 선별할 수 있게 되었습니다."}
    ]
}

print("JSONL 형식 예시:")
print("=" * 60)
print(json.dumps(training_example, ensure_ascii=False, indent=2))

### 고품질 학습 데이터의 특성

| 특성 | 설명 | 예시 |
|------|------|------|
| **일관성** | 동일한 형식/스타일 유지 | 모든 답변이 '핵심 요약' + '배경'으로 구성 |
| **다양성** | 다양한 입력 케이스 포함 | 긍정적/부정적 뉴스, 다양한 주제 |
| **정확성** | 정확한 정보, 오류 없음 | 팩트 체크된 답변 |
| **충분한 양** | 최소 50개 이상 권장 | 일반적으로 100-500개 |
| **적절한 길이** | 너무 길지도 짧지도 않게 | 입력과 출력의 균형 |

---

## 1.4 학습 데이터 준비하기

### 실습: AI 뉴스 요약 모델을 위한 데이터셋 만들기

IT/AI 뉴스 기사를 '핵심 요약'과 '주요 배경' 두 가지 항목으로 구조화하여 요약하는 모델을 위한 학습 데이터를 준비합니다.

In [ ]:
# 환경 설정
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()

# API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"OpenAI API Key 로드 완료: {api_key[:8]}...{api_key[-4:]}")
else:
    print("주의: OPENAI_API_KEY가 설정되지 않았습니다!")
    print(".env 파일 또는 환경변수에 API 키를 설정하세요.")

In [ ]:
# 파인튜닝용 샘플 AI 뉴스 데이터
training_data = [
    {
        "title": "데이터브릭스, 차세대 'AI 거버넌스' 프레임워크 '유니티 카탈로그 2.0' 발표",
        "content": "2025년 6월 20일, 데이터 및 AI 기업 데이터브릭스는 연례 컨퍼런스에서 차세대 데이터 거버넌스 프레임워크인 '유니티 카탈로그 2.0'을 공개했다. 이 프레임워크는 데이터, AI 모델, 관련 파이프라인 전체에 걸쳐 통합된 보안 및 거버넌스를 제공하는 것이 특징이다. 특히, 생성형 AI 모델의 출력 결과에 대한 실시간 모니터링과 유해성 탐지 기능을 탑재하여 기업이 책임감 있는 AI를 구현할 수 있도록 지원한다.",
        "source": "데이터 이코노미",
        "ideal_response": "- 핵심 요약: 데이터브릭스가 차세대 데이터 거버넌스 프레임워크 '유니티 카탈로그 2.0'을 발표했습니다.\n- 주요 배경: 데이터, AI 모델, 파이프라인 전체에 통합된 보안 및 거버넌스를 제공하고, 생성형 AI의 유해성 탐지 기능으로 책임감 있는 AI 구현을 지원하기 위함입니다."
    },
    {
        "title": "AI 옵저버빌리티 플랫폼 '뉴럴와처', 2억 달러 투자 유치",
        "content": "AI 모델의 운영 상태를 실시간으로 감시하고 분석하는 'AI 옵저버빌리티' 분야가 급성장하고 있다. 관련 스타트업 '뉴럴와처(NeuralWatcher)'는 최근 시리즈 C 펀딩에서 2억 달러 규모의 투자를 유치했다고 밝혔다. 뉴럴와처의 플랫폼은 LLM 애플리케이션에서 발생하는 데이터 드리프트, 성능 저하, 환각(Hallucination) 현상을 자동으로 탐지하고 개발자에게 경고한다.",
        "source": "AI 스타트업 위클리",
        "ideal_response": "- 핵심 요약: AI 옵저버빌리티 스타트업 '뉴럴와처'가 시리즈 C 펀딩에서 2억 달러를 유치했습니다.\n- 주요 배경: LLM 애플리케이션의 데이터 드리프트, 성능 저하, 환각 현상을 자동 탐지하여 AI 서비스의 신뢰도를 높이고 비즈니스 손실을 최소화하기 위함입니다."
    },
    {
        "title": "구글 클라우드, '버텍스 AI 에이전트 빌더' 정식 출시",
        "content": "구글 클라우드는 자사의 AI 플랫폼 버텍스 AI(Vertex AI)에 '에이전트 빌더' 기능을 정식으로 추가했다고 발표했다. 이 기능은 코딩 경험이 없는 사용자도 간단한 자연어 명령어와 그래픽 인터페이스를 통해 복잡한 AI 에이전트를 구축할 수 있도록 돕는다. 사용자는 지식 베이스를 연결하고, 구글 검색, 지도, 캘린더 등 다양한 구글 서비스를 도구로 추가할 수 있다.",
        "source": "클라우드 인사이트",
        "ideal_response": "- 핵심 요약: 구글 클라우드가 버텍스 AI에 '에이전트 빌더' 기능을 정식 출시했습니다.\n- 주요 배경: 코딩 없이 자연어와 그래픽 인터페이스로 AI 에이전트를 구축할 수 있어, 전문 개발자뿐 아니라 현업 실무자도 직접 AI 솔루션을 만들 수 있는 노코드 AI 시대를 열기 위함입니다."
    },
    {
        "title": "메타, 차세대 멀티모달 AI 모델 'LLaMA-Vision 3.0' 공개",
        "content": "메타는 텍스트, 이미지, 비디오를 동시에 처리할 수 있는 멀티모달 AI 모델 'LLaMA-Vision 3.0'을 공개했다. 이 모델은 기존 LLaMA 시리즈 대비 시각적 이해 능력이 40% 향상되었으며, 실시간 비디오 분석과 3D 객체 인식 기능을 지원한다. 특히 의료 영상 분석, 자율주행차 인식 시스템, AR/VR 콘텐츠 생성 등 다양한 산업 분야에서 활용 가능하다.",
        "source": "AI 테크 리뷰",
        "ideal_response": "- 핵심 요약: 메타가 텍스트, 이미지, 비디오를 동시에 처리하는 멀티모달 AI 모델 'LLaMA-Vision 3.0'을 공개했습니다.\n- 주요 배경: 기존 모델 대비 시각적 이해 능력이 40% 향상되었으며, 의료 영상 분석, 자율주행, AR/VR 등 다양한 산업에서 멀티모달 AI 활용이 필요했기 때문입니다."
    },
    {
        "title": "오픈AI, 'GPT-5' 개발 중단 발표... 대신 'GPT-4o Ultra' 집중 개발",
        "content": "오픈AI가 차세대 언어모델 'GPT-5' 개발을 중단하고, 대신 현재 GPT-4o의 성능을 극대화한 'GPT-4o Ultra' 개발에 집중한다고 발표했다. 이는 모델 크기를 무작정 키우는 것보다 효율성과 안전성을 우선시하는 전략 변화로 해석된다. GPT-4o Ultra는 기존 모델 대비 추론 속도가 3배 빨라지고, 에너지 효율성은 50% 개선될 예정이다.",
        "source": "AI 뉴스 데일리",
        "ideal_response": "- 핵심 요약: 오픈AI가 GPT-5 개발을 중단하고 'GPT-4o Ultra' 개발에 집중한다고 발표했습니다.\n- 주요 배경: 모델 크기 증가보다 효율성과 안전성을 우선시하는 전략 변화로, 추론 속도 3배 향상과 에너지 효율성 50% 개선을 통해 AI 발전의 새로운 패러다임을 제시하기 위함입니다."
    },
    {
        "title": "마이크로소프트, 'Azure AI Studio' 대폭 업데이트",
        "content": "마이크로소프트가 클라우드 기반 AI 개발 플랫폼 'Azure AI Studio'의 대규모 업데이트를 발표했다. 새로운 버전은 기업용 AI 애플리케이션 개발을 위한 통합 환경을 제공하며, 데이터 전처리부터 모델 배포까지 전 과정을 하나의 플랫폼에서 처리할 수 있다. 특히 '원클릭 AI 파이프라인' 기능으로 복잡한 MLOps 과정을 자동화하여 개발 시간을 80% 단축시킨다.",
        "source": "엔터프라이즈 IT",
        "ideal_response": "- 핵심 요약: 마이크로소프트가 'Azure AI Studio'의 대규모 업데이트를 발표했습니다.\n- 주요 배경: 기업용 AI 개발을 위한 통합 환경 제공과 '원클릭 AI 파이프라인'을 통해 MLOps를 자동화하여 개발 시간을 80% 단축하고 엔터프라이즈 AI의 진입 장벽을 낮추기 위함입니다."
    },
    {
        "title": "엔비디아, 'H200 Ultra' GPU 공개... AI 훈련 성능 2배 향상",
        "content": "엔비디아가 차세대 AI 전용 GPU 'H200 Ultra'를 공개했다. 이 GPU는 기존 H100 대비 AI 모델 훈련 성능이 2배 향상되었으며, 메모리 용량도 188GB로 확대되어 더 큰 규모의 모델 훈련이 가능하다. 특히 트랜스포머 아키텍처 기반 모델의 훈련에 최적화되어 있어, LLM 개발 시간을 크게 단축시킬 수 있다. 또한 새로운 'NVLink 5.0' 기술로 GPU 간 데이터 전송 속도가 3배 빨라졌다.",
        "source": "하드웨어 인사이트",
        "ideal_response": "- 핵심 요약: 엔비디아가 AI 전용 GPU 'H200 Ultra'를 공개했습니다.\n- 주요 배경: 기존 H100 대비 훈련 성능 2배 향상, 메모리 188GB 확대, NVLink 5.0으로 데이터 전송 속도 3배 향상을 통해 대규모 LLM 개발을 위한 핵심 인프라를 구축하기 위함입니다."
    },
    {
        "title": "스탠포드 대학, AI 안전성 벤치마크 'SafetyBench 2.0' 발표",
        "content": "스탠포드 대학 AI 연구소가 AI 모델의 안전성을 종합적으로 평가하는 새로운 벤치마크 'SafetyBench 2.0'을 발표했다. 이 벤치마크는 편향성, 독성, 환각, 개인정보 유출 등 12개 카테고리에서 AI 모델의 위험성을 정량적으로 측정한다. 기존 평가 방식과 달리 실제 사용 환경을 모방한 시나리오 기반 테스트를 도입하여 더욱 현실적인 평가가 가능하다.",
        "source": "AI 안전성 저널",
        "ideal_response": "- 핵심 요약: 스탠포드 대학이 AI 안전성 평가 벤치마크 'SafetyBench 2.0'을 발표했습니다.\n- 주요 배경: 편향성, 독성, 환각, 개인정보 유출 등 12개 카테고리에서 실제 사용 환경을 모방한 시나리오 기반 테스트로 AI 모델의 위험성을 정량적으로 측정하여 AI 안전성 연구의 중요성을 부각시키기 위함입니다."
    },
    {
        "title": "테슬라, FSD v13 업데이트 배포... 도심 주행 성공률 95% 달성",
        "content": "테슬라가 완전 자율주행 소프트웨어 'FSD v13' 업데이트를 배포했다. 이번 버전은 복잡한 도심 환경에서의 주행 성공률이 95%에 달해, 상용화 수준에 근접했다는 평가를 받고 있다. 특히 교차로 좌회전, 보행자 회피, 공사 구간 우회 등 까다로운 상황에서의 판단 능력이 크게 향상되었다. 테슬라는 2025년 말까지 완전 무인 택시 서비스를 시작할 계획이다.",
        "source": "자율주행 테크",
        "ideal_response": "- 핵심 요약: 테슬라가 FSD v13 업데이트를 배포하여 도심 주행 성공률 95%를 달성했습니다.\n- 주요 배경: 교차로 좌회전, 보행자 회피, 공사 구간 우회 등 까다로운 상황에서의 판단 능력 향상을 통해 2025년 말 완전 무인 택시 서비스 시작을 목표로 하는 교통 혁명을 준비하기 위함입니다."
    },
    {
        "title": "IBM, 양자-AI 융합 플랫폼 'Quantum-AI Hybrid' 공개",
        "content": "IBM이 양자 컴퓨팅과 인공지능을 결합한 혁신적인 플랫폼 'Quantum-AI Hybrid'를 공개했다. 이 플랫폼은 양자 컴퓨터의 병렬 처리 능력과 AI의 패턴 인식 능력을 결합하여, 기존 컴퓨터로는 해결하기 어려운 복잡한 최적화 문제를 해결한다. 특히 신약 개발, 금융 리스크 분석, 물류 최적화 등의 분야에서 기존 방식 대비 1000배 빠른 처리 속도를 보여준다.",
        "source": "퀀텀 컴퓨팅 리뷰",
        "ideal_response": "- 핵심 요약: IBM이 양자 컴퓨팅과 AI를 결합한 'Quantum-AI Hybrid' 플랫폼을 공개했습니다.\n- 주요 배경: 양자 컴퓨터의 병렬 처리와 AI의 패턴 인식을 결합하여 신약 개발, 금융 리스크, 물류 최적화 등에서 기존 대비 1000배 빠른 처리로 컴퓨팅 패러다임의 근본적 변화를 가져오기 위함입니다."
    }
]

print(f"총 {len(training_data)}개의 학습 데이터 준비 완료")
print("\n데이터 미리보기:")
for i, data in enumerate(training_data[:3], 1):
    print(f"\n[{i}] {data['title'][:40]}...")

---

## 1.5 JSONL 파일 생성하기

### 학습 데이터를 JSONL 형식으로 변환

In [ ]:
import json
import os

# 시스템 메시지 정의
system_message = "당신은 IT 전문 기자입니다. 주어진 뉴스 기사를 '핵심 요약'과 '주요 배경'으로 나누어 구조화된 형식으로 요약해야 합니다."

# JSONL 데이터 생성
jsonl_data = []

for article in training_data:
    user_prompt = f"다음 뉴스 기사를 요약해 주세요:\n\n제목: {article['title']}\n내용: {article['content']}"
    
    jsonl_data.append({
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": article['ideal_response']}
        ]
    })

# 파일 저장
os.makedirs("datasets", exist_ok=True)
file_path = "datasets/news_finetuning_data.jsonl"

with open(file_path, "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"JSONL 파일 생성 완료: {file_path}")
print(f"총 {len(jsonl_data)}개의 학습 예시 포함")

In [ ]:
# 생성된 JSONL 파일 확인
print("생성된 JSONL 파일 내용 (처음 2줄):")
print("=" * 70)

with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 2:
            break
        data = json.loads(line)
        print(f"\n[예시 {i+1}]")
        print(f"System: {data['messages'][0]['content'][:50]}...")
        print(f"User: {data['messages'][1]['content'][:50]}...")
        print(f"Assistant: {data['messages'][2]['content'][:50]}...")

---

# Part 2: 심화

---

## 2.1 OpenAI 파인튜닝 프로세스

### 파인튜닝 4단계

| 단계 | 설명 | 소요 시간 |
|------|------|----------|
| **Step 1** | 데이터셋 준비 (JSONL) | 수 시간~수 일 |
| **Step 2** | 파일 업로드 | 수 초 |
| **Step 3** | 파인튜닝 작업 생성 및 실행 | 수 분~수 시간 |
| **Step 4** | 파인튜닝된 모델 사용 | 즉시 |

### 지원 모델 (2025년 1월 기준)

| 모델 | 특징 | 파인튜닝 비용 |
|------|------|-------------|
| gpt-4o-2024-08-06 | 최신 플래그십 | $25/1M 토큰 |
| gpt-4o-mini-2024-07-18 | 비용 효율적 | $3/1M 토큰 |
| gpt-4.1-mini-2025-04-14 | 최신 경량 모델 | $3/1M 토큰 |
| gpt-3.5-turbo-0125 | 레거시 | $8/1M 토큰 |

In [ ]:
# OpenAI 클라이언트 초기화
from openai import OpenAI

client = OpenAI()
print("OpenAI 클라이언트 초기화 완료!")

### Step 2: 학습 데이터 업로드

In [ ]:
# 파일 업로드
training_file = client.files.create(
    file=open(file_path, "rb"),
    purpose="fine-tune"
)

print(f"파일 업로드 완료!")
print(f"파일 ID: {training_file.id}")
print(f"파일명: {training_file.filename}")
print(f"상태: {training_file.status}")

### Step 3: 파인튜닝 작업 생성

In [ ]:
# 파인튜닝 작업 생성
# 참고: gpt-4o-mini 또는 gpt-4.1-mini 사용 권장 (비용 효율적)
target_model = "gpt-4o-mini-2024-07-18"  # 또는 "gpt-4.1-mini-2025-04-14"

fine_tuning_job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model=target_model,
    # hyperparameters={  # 선택적 하이퍼파라미터
    #     "n_epochs": 3,
    #     "learning_rate_multiplier": 1.0
    # }
)

job_id = fine_tuning_job.id
print(f"파인튜닝 작업 생성 완료!")
print(f"작업 ID: {job_id}")
print(f"상태: {fine_tuning_job.status}")
print(f"기반 모델: {target_model}")

---

## 2.2 파인튜닝 작업 모니터링

### 작업 상태 확인

파인튜닝 작업은 데이터 크기에 따라 수 분에서 수 시간까지 소요될 수 있습니다.

In [ ]:
# 작업 상태 확인 함수
def check_finetuning_status(job_id):
    """파인튜닝 작업 상태 확인"""
    job_info = client.fine_tuning.jobs.retrieve(job_id)
    
    print(f"작업 ID: {job_info.id}")
    print(f"상태: {job_info.status}")
    print(f"기반 모델: {job_info.model}")
    
    if job_info.finished_at:
        print(f"완료 시간: {job_info.finished_at}")
    
    if job_info.fine_tuned_model:
        print(f"파인튜닝된 모델: {job_info.fine_tuned_model}")
    
    if job_info.error:
        print(f"에러: {job_info.error}")
    
    return job_info

# 현재 상태 확인
job_info = check_finetuning_status(job_id)

In [ ]:
# 자동 모니터링 (작업 완료까지 대기)
import time

def wait_for_finetuning(job_id, check_interval=60):
    """파인튜닝 작업 완료까지 대기"""
    print(f"파인튜닝 작업 모니터링 시작 (매 {check_interval}초마다 확인)...")
    print("=" * 50)
    
    while True:
        job_info = client.fine_tuning.jobs.retrieve(job_id)
        status = job_info.status
        
        print(f"[{time.strftime('%H:%M:%S')}] 상태: {status}")
        
        if status == "succeeded":
            print("\n파인튜닝 완료!")
            print(f"새로운 모델 ID: {job_info.fine_tuned_model}")
            return job_info.fine_tuned_model
        
        elif status == "failed":
            print("\n파인튜닝 실패!")
            print(f"에러: {job_info.error}")
            return None
        
        elif status == "cancelled":
            print("\n파인튜닝 취소됨")
            return None
        
        time.sleep(check_interval)

# 주의: 이 셀은 파인튜닝이 완료될 때까지 대기합니다 (수 분~수 시간)
# 테스트 시에는 주석 처리하고, 수동으로 상태를 확인하세요

# fine_tuned_model_id = wait_for_finetuning(job_id, check_interval=60)

In [ ]:
# 파인튜닝 이벤트 로그 확인
events = client.fine_tuning.jobs.list_events(job_id, limit=10)

print("파인튜닝 이벤트 로그 (최근 10개):")
print("=" * 60)
for event in events.data:
    print(f"[{event.created_at}] {event.message}")

---

## 2.3 파인튜닝된 모델 사용하기

### 파인튜닝 완료 후 모델 ID 확인

In [ ]:
# 완료된 파인튜닝 작업에서 모델 ID 가져오기
job_info = client.fine_tuning.jobs.retrieve(job_id)

if job_info.status == "succeeded":
    fine_tuned_model_id = job_info.fine_tuned_model
    print(f"파인튜닝된 모델 ID: {fine_tuned_model_id}")
else:
    print(f"작업이 아직 완료되지 않았습니다. 현재 상태: {job_info.status}")
    fine_tuned_model_id = None  # 또는 이전에 완료된 모델 ID를 직접 입력

In [ ]:
# 이전에 파인튜닝한 모델이 있다면 직접 ID 입력
# fine_tuned_model_id = "ft:gpt-4o-mini-2024-07-18:your-org::xxxxx"

if fine_tuned_model_id:
    print(f"사용할 모델: {fine_tuned_model_id}")
else:
    print("파인튜닝된 모델 ID가 없습니다. 작업 완료를 기다리거나 ID를 직접 입력하세요.")

### 기본 모델 vs 파인튜닝 모델 비교

In [ ]:
# 테스트용 새로운 뉴스 기사
test_article = {
    "title": "메타, 차세대 언어 모델 '라마 4' 공개... 온디바이스 AI 시장 정조준",
    "content": "페이스북의 모회사 메타가 최신 거대 언어 모델(LLM)인 '라마 4(Llama 4)'를 공개했습니다. 라마 4는 이전 모델보다 크기는 줄이면서도 성능은 대폭 향상시킨 것이 특징입니다. 특히, 스마트폰과 같은 개인 기기에서 직접 구동되는 온디바이스 AI 환경에 최적화되어, 인터넷 연결 없이도 빠른 속도로 고품질의 AI 기능을 제공할 수 있습니다. 메타는 라마 4를 오픈소스로 공개하여, 더 많은 개발자들이 AI 기술에 접근하고 혁신을 가속화할 수 있도록 지원할 것이라고 밝혔습니다."
}

test_prompt = f"다음 뉴스 기사를 요약해 주세요:\n\n제목: {test_article['title']}\n내용: {test_article['content']}"

print("테스트 기사:")
print(f"제목: {test_article['title']}")
print(f"내용: {test_article['content'][:100]}...")

In [ ]:
# 기본 모델 응답
print("=" * 70)
print("1. 기본 모델 (gpt-4o-mini) 응답")
print("=" * 70)

base_response = client.chat.completions.create(
    model=target_model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": test_prompt}
    ],
    temperature=0.3
)

print(base_response.choices[0].message.content)

In [ ]:
# 파인튜닝된 모델 응답
if fine_tuned_model_id:
    print("=" * 70)
    print("2. 파인튜닝된 모델 응답")
    print("=" * 70)
    
    # 파인튜닝된 모델은 시스템 메시지 없이도 학습된 형식으로 응답
    ft_response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=[
            {"role": "user", "content": test_prompt}
        ],
        temperature=0.3
    )
    
    print(ft_response.choices[0].message.content)
else:
    print("파인튜닝된 모델이 없어 비교할 수 없습니다.")

---

## 2.4 하이퍼파라미터 조정하기

### 주요 하이퍼파라미터

| 파라미터 | 설명 | 기본값 | 권장 범위 |
|---------|------|--------|----------|
| `n_epochs` | 학습 반복 횟수 | 자동 | 1-10 |
| `learning_rate_multiplier` | 학습률 배수 | 자동 | 0.1-2.0 |
| `batch_size` | 배치 크기 | 자동 | 1-64 |

In [ ]:
# 하이퍼파라미터를 지정한 파인튜닝 (예시)
# 주의: 실제 실행 시 비용이 발생합니다

# fine_tuning_job_custom = client.fine_tuning.jobs.create(
#     training_file=training_file.id,
#     model="gpt-4o-mini-2024-07-18",
#     hyperparameters={
#         "n_epochs": 3,  # 학습 반복 횟수
#         "learning_rate_multiplier": 1.5,  # 학습률 배수
#     }
# )

print("하이퍼파라미터 조정 예시:")
print("- n_epochs: 3 (기본: 자동)")
print("- learning_rate_multiplier: 1.5 (기본: 자동)")
print("\n주의: 실제 실행 시 추가 비용이 발생합니다.")

### 하이퍼파라미터 조정 가이드

| 상황 | 권장 조정 |
|------|----------|
| 과적합 (train loss 낮고 val loss 높음) | `n_epochs` 감소, `learning_rate_multiplier` 감소 |
| 과소적합 (train loss 높음) | `n_epochs` 증가, `learning_rate_multiplier` 증가 |
| 데이터가 적을 때 (50개 미만) | `n_epochs` 증가, `learning_rate_multiplier` 감소 |
| 데이터가 많을 때 (1000개 이상) | `n_epochs` 감소 (3 이하) |

---

## 2.5 실무 활용 및 비용 최적화

### 파인튜닝 비용 구조

```
총 비용 = 학습 비용 + 추론 비용

학습 비용:
  - gpt-4o-mini: $3 / 1M 학습 토큰
  - 예: 100개 예시 x 500 토큰 x 3 에포크 = 150K 토큰 = $0.45

추론 비용 (파인튜닝된 모델):
  - gpt-4o-mini: 입력 $0.30 / 출력 $1.20 per 1M 토큰
  - 기본 모델보다 약간 비싸지만, 짧은 프롬프트로 상쇄 가능
```

### 비용 최적화 전략

| 전략 | 설명 |
|------|------|
| 작은 모델 선택 | gpt-4o보다 gpt-4o-mini 사용 |
| 데이터 품질 > 양 | 50개 고품질 > 500개 저품질 |
| 에포크 조정 | 과적합 방지로 학습 비용 절감 |
| 짧은 프롬프트 | 학습 후 시스템 메시지 생략 가능 |

In [ ]:
# 비용 추정 함수
def estimate_finetuning_cost(num_examples, avg_tokens_per_example, epochs=3):
    """파인튜닝 비용 추정 (gpt-4o-mini 기준)"""
    training_cost_per_1m = 3.0  # $3 per 1M tokens
    
    total_tokens = num_examples * avg_tokens_per_example * epochs
    estimated_cost = (total_tokens / 1_000_000) * training_cost_per_1m
    
    print(f"파인튜닝 비용 추정 (gpt-4o-mini):")
    print(f"  - 예시 수: {num_examples}개")
    print(f"  - 예시당 평균 토큰: {avg_tokens_per_example}")
    print(f"  - 에포크: {epochs}")
    print(f"  - 총 학습 토큰: {total_tokens:,}")
    print(f"  - 예상 비용: ${estimated_cost:.2f}")
    
    return estimated_cost

# 현재 데이터셋 비용 추정
estimate_finetuning_cost(len(training_data), 500, 3)

### 파인튜닝 모델 관리

In [ ]:
# 파인튜닝된 모델 목록 확인
models = client.models.list()

print("파인튜닝된 모델 목록:")
print("=" * 60)

ft_models = [m for m in models.data if m.id.startswith("ft:")]

if ft_models:
    for model in ft_models:
        print(f"  - {model.id}")
        print(f"    생성일: {model.created}")
else:
    print("  파인튜닝된 모델이 없습니다.")

In [ ]:
# 파인튜닝된 모델 삭제 (필요시)
# 주의: 삭제된 모델은 복구할 수 없습니다

# if fine_tuned_model_id:
#     client.models.delete(fine_tuned_model_id)
#     print(f"모델 삭제됨: {fine_tuned_model_id}")

print("모델 삭제 예시 (주석 처리됨):")
print("client.models.delete(model_id)")

---

## 실습 퀴즈

**난이도 분포**: 기본 3개 / 응용 3개 / 복합 2개 / 종합 2개

---

### Q1. 파인튜닝 개념 이해 (기본)

**문제**: 다음 빈칸을 채우세요.

1. 파인튜닝은 모델의 내부 (______)을 업데이트하는 과정입니다.
2. 새로운 '지식'을 추가하려면 (______)가 적합하고, '행동 방식'을 바꾸려면 (______)가 적합합니다.
3. OpenAI 파인튜닝 데이터는 (______) 형식으로 준비해야 합니다.

In [ ]:
# 여기에 답을 작성하세요
answer1 = ""
answer2_knowledge = ""
answer2_behavior = ""
answer3 = ""

print(f"1. {answer1}")
print(f"2. 지식: {answer2_knowledge}, 행동: {answer2_behavior}")
print(f"3. {answer3}")

### Q2. JSONL 형식 작성 (기본)

**문제**: 다음 정보를 JSONL 형식으로 변환하세요.

- 시스템 메시지: "당신은 친절한 고객 상담원입니다."
- 사용자 질문: "환불은 어떻게 하나요?"
- 이상적 답변: "환불은 구매일로부터 7일 이내에 가능합니다. 마이페이지에서 신청하실 수 있습니다."

In [ ]:
import json

# 여기에 코드를 작성하세요


### Q3. 적합한 기술 선택 (기본)

**문제**: 다음 각 상황에 가장 적합한 기술(프롬프트 엔지니어링, RAG, 파인튜닝)을 선택하세요.

1. 회사의 최신 제품 정보를 기반으로 고객 질문에 답변하는 챗봇
2. 모든 마케팅 문구를 특정 브랜드 톤앤매너로 작성하는 봇
3. 간단한 텍스트 번역 작업

In [ ]:
# 여기에 답을 작성하세요
scenario_1 = ""  # RAG / Fine-tuning / Prompting
scenario_2 = ""
scenario_3 = ""

print(f"1. 최신 제품 정보 챗봇: {scenario_1}")
print(f"2. 브랜드 톤앤매너 유지: {scenario_2}")
print(f"3. 간단한 번역: {scenario_3}")

### Q4. 학습 데이터 생성 (응용)

**문제**: "친근한 말투의 AI 비서"를 파인튜닝하기 위한 학습 데이터 3개를 JSONL 형식으로 작성하세요. 각 예시는 다른 주제(날씨, 일정, 추천)여야 합니다.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 파인튜닝 작업 관리 (응용)

**문제**: 다음 작업을 수행하는 코드를 작성하세요.

1. 현재 진행 중인 파인튜닝 작업 목록 확인
2. 가장 최근 작업의 상태와 이벤트 로그 출력

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 비용 계산 (응용)

**문제**: 다음 조건의 파인튜닝 비용을 계산하세요.

- 학습 데이터: 200개 예시
- 예시당 평균 토큰: 800
- 에포크: 5
- 모델: gpt-4o-mini ($3/1M 학습 토큰)

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 데이터 품질 검증 (복합)

**문제**: JSONL 파일을 읽어 다음을 검증하는 함수를 작성하세요.

1. 각 줄이 유효한 JSON인지 확인
2. 필수 키(messages)가 있는지 확인
3. 각 메시지에 role과 content가 있는지 확인
4. 총 예시 수와 평균 토큰 수 출력

In [ ]:
def validate_jsonl(file_path):
    """JSONL 파일 품질 검증"""
    # 여기에 코드를 작성하세요
    pass

# 테스트
# validate_jsonl("datasets/news_finetuning_data.jsonl")

### Q8. 모델 비교 함수 (복합)

**문제**: 기본 모델과 파인튜닝된 모델의 응답을 비교하는 함수를 작성하세요.

요구사항:
1. 동일한 프롬프트로 두 모델 호출
2. 응답 내용, 응답 시간, 토큰 사용량 비교
3. 결과를 딕셔너리로 반환

In [ ]:
import time

def compare_models(prompt, base_model, finetuned_model, system_message=None):
    """기본 모델과 파인튜닝 모델 비교"""
    # 여기에 코드를 작성하세요
    pass

# 테스트
# result = compare_models(
#     "AI의 미래에 대해 설명해주세요.",
#     "gpt-4o-mini",
#     fine_tuned_model_id
# )

### Q9. 도메인 특화 데이터셋 생성 (종합)

**문제**: 특정 도메인(예: 법률, 의료, 금융 중 선택)에 특화된 Q&A 데이터셋을 5개 이상 생성하고, JSONL 파일로 저장하세요.

요구사항:
1. 일관된 답변 형식 유지
2. 도메인 전문 용어 포함
3. 다양한 질문 유형

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 파인튜닝 파이프라인 구축 (종합)

**문제**: 파인튜닝 전체 파이프라인을 클래스로 구현하세요.

요구 기능:
1. `prepare_data()`: 데이터 준비 및 JSONL 생성
2. `upload_and_train()`: 파일 업로드 및 파인튜닝 시작
3. `monitor()`: 작업 상태 모니터링
4. `evaluate()`: 파인튜닝된 모델 평가

In [ ]:
class FineTuningPipeline:
    """OpenAI 파인튜닝 파이프라인"""
    
    def __init__(self, model_name="gpt-4o-mini-2024-07-18"):
        self.client = OpenAI()
        self.model_name = model_name
        self.training_file_id = None
        self.job_id = None
        self.finetuned_model_id = None
    
    def prepare_data(self, data, output_path):
        """학습 데이터 준비 및 JSONL 저장"""
        # 여기에 코드를 작성하세요
        pass
    
    def upload_and_train(self, file_path, hyperparameters=None):
        """파일 업로드 및 파인튜닝 시작"""
        # 여기에 코드를 작성하세요
        pass
    
    def monitor(self, check_interval=60):
        """파인튜닝 작업 모니터링"""
        # 여기에 코드를 작성하세요
        pass
    
    def evaluate(self, test_prompts):
        """파인튜닝된 모델 평가"""
        # 여기에 코드를 작성하세요
        pass

# 테스트
# pipeline = FineTuningPipeline()
# pipeline.prepare_data(training_data, "datasets/custom_data.jsonl")

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| 파인튜닝 | 모델의 내부 가중치를 업데이트 | 스타일/톤/형식 일관성 |
| Prompting vs RAG vs FT | 각각 다른 문제 해결 | 상황에 맞는 기술 선택 |
| JSONL 형식 | messages 배열 (system, user, assistant) | 학습 데이터 준비 |
| 데이터 품질 | 일관성, 다양성, 정확성 | 최소 50개 이상 권장 |
| 파일 업로드 | client.files.create() | purpose="fine-tune" |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 기법 | 권장 설정 |
|------|----------|----------|
| 파인튜닝 API | client.fine_tuning.jobs.create() | gpt-4o-mini 권장 |
| 모니터링 | jobs.retrieve(), list_events() | 60초 간격 체크 |
| 하이퍼파라미터 | n_epochs, learning_rate_multiplier | 자동 설정 우선 |
| 비용 최적화 | 작은 모델, 품질 데이터, 짧은 프롬프트 | $3/1M 토큰 (학습) |

### 파인튜닝 체크리스트

```
1. 사전 준비:
   [ ] 파인튜닝이 적합한 상황인지 확인
   [ ] 고품질 예시 데이터 50개 이상 준비
   [ ] 일관된 형식/스타일 유지
   
2. 데이터 준비:
   [ ] JSONL 형식으로 변환
   [ ] messages 구조 검증 (system, user, assistant)
   [ ] 토큰 수 확인 (너무 길지 않게)
   
3. 파인튜닝 실행:
   [ ] 파일 업로드 (purpose="fine-tune")
   [ ] 작업 생성 (모델, 하이퍼파라미터)
   [ ] 상태 모니터링
   
4. 평가 및 배포:
   [ ] 기본 모델과 성능 비교
   [ ] 다양한 테스트 케이스 검증
   [ ] 프로덕션 적용
```

### 실무 팁

1. **RAG vs 파인튜닝**: 지식 추가는 RAG, 행동 변화는 파인튜닝
2. **데이터 품질이 핵심**: 50개 고품질 > 500개 저품질
3. **작은 모델부터**: gpt-4o-mini로 시작, 필요시 gpt-4o로 업그레이드
4. **시스템 메시지 생략**: 파인튜닝 후에는 짧은 프롬프트로 충분
5. **비용 모니터링**: 학습 비용 + 추론 비용 함께 고려